# Setting up hadoop cluster


Setting up environment:

1. Oracle Box: Import Appliance, assign 1 CPU and 2GB memory. Save the imported VMs to a specific address on your drive by setting it up in base.
2. Don't forget to "reinitialize the MAC address for all network cards." (In Virtual Box version 6 and higher it is "Generate new MAC addresses for all network adapters.")
3. After importing them and renaming their group, change their network setting to host only.
4. These three VMs have two users: 
root with pass root
hadoop with pass hadoop
5. Check you host (in my case windows) IP by going to control panel>network and sharing center > Change adapter settign > Check ethernet 5 > properties > click on IPV4 and see your IP. In my case 192.168.56.1.
6. Back in master node (CentOS vm) open terminal from Applications? Favorites> Terminal
7. In terminal type nmtui to set the IP: 
disable IPV6;
make IPV4 manual and set it in the same IP range as your host machine;
set a host name: in this case node-master
8. in terminal type ifconfig to see if IP is set properly.
9. type exit to exit terminal.
10. In wired settings disable DNS and route.
11. From node-master ping host machine: ping 192.168.56.1
12. From host machine ping node-master: ping 192.168.56.10
13. Repeat steps 3 to 12 for all three nodes.
14. Open ssh terminal by mobaxterm to each of your machines. SSH port is 22.
15. To allow each node ping the other one with alias, we need to setup FQDN (Fully Qualified Domain Name). Consider that all nodes can see each other, but nodes do not see each others' names. To resolve this in windows: 

C:\Windows\System32\drivers\etc\hosts

Then edit hosts like below:

```
192.168.56.2 Node2
192.168.1.1 PC1 (This is my own laptop.)
192.168.56.10 node-master
192.168.56.11 node1
192.168.56.12 node2
```

This is not necessary for our windows machine (host), but for our nodes we need to do it.

On Linux nodes FQDN can set up on /etc/hosts
You may need to su to root user:
    ```
+ su
+ enter password for root
+ cd .. to get to root
+ cd etc
+ vi hosts

Hit insert button on key board to be able to add lines to hosts file. Add
```
192.168.56.10 node-master
192.168.56.11 node1
192.168.56.12 node2
```

Copy this file to all the other nodes or repeat step 15 for all of them.

To edit with vi, hit insert to be able to edit the file. To quit without saving use Esc, then type :q! To save and quit hit Esc, then type :wq (Write and Quit).

16. You can ssh from one node to another. For example on ndoe-master you can:
+ ssh node1
Enter password and you are connected to node1.

With user hadoop, you will be in /home/hadoop.

17. To connect passwordless to your servers:
I want user hadoop from node-master be able to connect to ndoe1 and node2 with ssh without password.
To do so (passwordless ssh):
+ open a session to node-master with the user you want to connect (in this case, hadoop).
+ On node-master with user hadoop generate a key:
+ ssh-keygen -b 4096
4096 is number of bits of the key.
The generated key by default is at /home/hadoop/.ssh/id_rsa. You can cat it.
cat /home/hadoop/.ssh/id_rsa.pub

18- Copy the generated key (be carfeul, just public key) to all other nodes. This includes master as well.
+ ssh-copy-id -i /home/hadoop/.ssh/id_rsa.pub hadoop@node-master

For the first time it will ask for password, but afterwards you can ssh passwordless.

19- Repeat step 18 for node1 and node2:
+ ssh-copy-id -i /home/hadoop/.ssh/id_rsa.pub hadoop@node1

If you have n nodes, you have to do it for all of them.

20- For nodes to be able to communicate with each other they need to see each others' IP and ports. At this stage we need to open firewalls.

Add the ports to firewall:

In hadoop 3 we need to open 9870, 9871, 9866, 9867 etc for different services of hadoop. We need to do it on each server for each port.

+ sudo firewall-cmd —zone=public —add-port=9000/tcp —permanent
+ sudo firewall-cmd reload

The ports we need to open for the beginning are as follows:

```
namenode HTTP Port: 9870
Namenode HTTPS Port: 9871
Datanode HTTP Port: 9864
Datanode HTTPS Port: 9865
SecondaryNN HTTP Port: 9868
SecoondaryNN HTTPS Port: 9869
```

## Changing replication factor and block size for individual files

It is possible to change file relication factor and block size while putting a file into hdfs.

```
hdfs dfs -Ddfs.blocksize=256000000 -put /home/hadoop/file.xml /xmlfile
hdfs dfs -setrep 2 /data/retail/file.csv
```

In specific files and direcotries where data volume is low and it is necessary to be available on all servers, like spark config files, we need to increase data locality and put it on all nodes. We don' have to, but it is a good idea to increase replication factor.
```
hdfs dfs -setrep 95 /data/file.xml
```


## Changing user access

| User | Group | Other|
|----------|:-------------:|------:|
| rwx  | rwx   | rwx |

+ r: read
+ w: write
+ x: execute

+ To enable security change hdfs-site.xml
```
dfs,namenode.acls.enabled
hdfs getconf -confskey dfs.namenode.acls.enabled
```

Change dfs.namenode.acls.enabled to true in hdfs-site.xml. As usual copy this on all nodes.
```
scp /opt/hadoop/etc/hadoop/*.xml node1:/opt/hadoop/etc/hadoop/
scp /opt/hadoop/etc/hadoop/*.xml node2:/opt/hadoop/etc/hadoop/

```

Stop and start hdfs:

```
stop-dfs.sh
start-dfs.sh
```

+ I want to give user dr.who read write and execute access to directory data/crm.

```
hdfs dfs -setfacl --set user::---,user:hadoop:rwx,user:dr.who:rwx,group::r--,other::r-- /data/crm

```

+ -setfacl: set file access control list
+ You can check your user with whoami.


### hdfs-site.xml

```
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
  
  <property>
    <name>dfs.replication</name>
    <value>1</value>
  </property>
  
  <property>
    <name>dfs.datanode.data.dir</name>
    <value>/opt/data</value>
  </property>
  
    <property>
    <name>dfs.namenode.name.dir</name>
    <value>/opt/data</value>
  </property>
  
  <property>
    <name> dfs.namenode.acls.enabled </name>
    <value> true </value>
  </property> 
  

</configuration>
```

## Nodes status

Each node is available at http://node1:9864/datanode.html

+ hadoop UI port: 9870
+ data node's port: 9864

## Creating new user and groups

+ The following users are created at os level:

```
sudo groupadd datasxience
sudo useradd -m user1
sudo passwd user1
sudo usermod -aG datascience user1
```

passwd sets a password. Here I set password "user1" for user "user1".

Now user1 is added to group datascience. 

+ This user must be created on both master (namenode) and all datanodes.
+ To make sure user1 is created successfully switch user at os level:

```
su user1
```

+ granting access to use1

```
hdfs dfs -setfacl --set user::---,user:hadoop:rwx,user:user1:rwx,user:dr.who:r--,group::---,other::--- /datascience
```

Now that user1 has write access to directory /datascience we can switch user and write inside it:

```
su user1
hdfs dfs -put  /opt/hadoop/testdata/file.txt /datascience
```

Make sureuser1 also has HJava and hadoop home set in .bashrc file in /home/user1.


### Q1: How to change user in mobaxtem to see the GUI part?

## .bashrc file of user1

```
export JAVA_HOME=/opt/java
PATH=$JAVA_HOME/bin:$PATH
export PATH

export HADOOP_HOME=/opt/hadoop
PATH=$HADOOP_HOME/bin:$HADOOP_HOME/sbin:$PATH
export PATH
```

+ Copy .bashrc from node-master to node1 and node2 (datanodes). 
+ source .bashrc file on all nodes to apply changes.

```
source .bashrc
```

**Note that if you set .bashrc in /etc/.bashrc you don't need to set it for each user.**

```
hdfs dfs -mkdir ds
```

## Storage policy or tierd storage

+ Tape
+ SSD
+ SAN
+ NAS

Stoage policies:
+ Hot
+ Cold
+ Warm
+ All_SSD
+ One_SSD
+ Lazy_Persit

Infrastructure team provides new directories and locations to us.

+ For setting policies, make sure dfs.storage.policy.enabled is true. By default it is true. 
+ Copy the new hdfs-site.xml file to all data nodes.


After making changes to hdfs-site.xml and copy it to all datanodes, stop-dfs.sh and start-dfs.sh to apply the chnages.

```
hdfs storagepolicies -listPolicies
```

Output:
    ```
    lock Storage Policies:
        BlockStoragePolicy{PROVIDED:1, storageTypes=[PROVIDED, DISK], creationFallbacks=[PROVIDED, DISK], replicationFallbacks=[PROVIDED, DISK]}
        BlockStoragePolicy{COLD:2, storageTypes=[ARCHIVE], creationFallbacks=[], replicationFallbacks=[]}
        BlockStoragePolicy{WARM:5, storageTypes=[DISK, ARCHIVE], creationFallbacks=[DISK, ARCHIVE], replicationFallbacks=[DISK, ARCHIVE]}
        BlockStoragePolicy{HOT:7, storageTypes=[DISK], creationFallbacks=[], replicationFallbacks=[ARCHIVE]}
        BlockStoragePolicy{ONE_SSD:10, storageTypes=[SSD, DISK], creationFallbacks=[SSD, DISK], replicationFallbacks=[SSD, DISK]}
        BlockStoragePolicy{ALL_SSD:12, storageTypes=[SSD], creationFallbacks=[DISK], replicationFallbacks=[DISK]}
        BlockStoragePolicy{LAZY_PERSIST:15, storageTypes=[RAM_DISK, DISK], creationFallbacks=[DISK], replicationFallbacks=[DISK]}

    ```